In [1]:
import cv2
from google.colab.patches import cv2_imshow

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
!pip install mediapipe opencv-python matplotlib

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 36.1/36.1 MB 34.5 MB/s eta 0:00:00


#  Trim and Stanrdardize
key points of the first step

Trim:


*   Removes empty/static frames from start and end of videos
*   Uses motion detection to find meaningful action segments
*   Keeps 5 frames before first motion and 5 frames after last motion


Standardize:


*   Calculates mean frame count across all videos
*   resamples each video to match this mean frame count
*   All videos end up with same number of frames: 113 frame



In [4]:
import cv2
import numpy as np
from pathlib import Path
from tqdm import tqdm

def process_sign_language_videos(input_folder, output_base_folder):
    """
    Process sign language videos with error tracking
    Parameters:
    - input_folder: Path to folder containing input videos
    - output_base_folder: Path to base folder where processed videos will be saved
    Returns:
    - processed_folder: Path object pointing to output folder
    - failed_videos: List of tuples (video_path, error_message)
    - successful_videos: List of successfully processed video paths
    """
    # Convert input path to Path object
    input_folder = Path(input_folder)
    output_base_folder = Path(output_base_folder)

    # Create output folders with new name
    output_folder = output_base_folder / 'processed_videos_v2'
    output_folder.mkdir(parents=True, exist_ok=True)

    # Track processing status
    failed_videos = []
    successful_videos = []

    # Find all MP4 files
    video_files = list(input_folder.rglob('*.mp4'))
    print(f"Found {len(video_files)} videos")

    # First calculate mean video length
    print("Calculating average video length...")
    frame_counts = []
    for video_path in tqdm(video_files):
        try:
            cap = cv2.VideoCapture(str(video_path))
            frame_counts.append(int(cap.get(cv2.CAP_PROP_FRAME_COUNT)))
            cap.release()
        except Exception as e:
            failed_videos.append((video_path, f"Error during frame count: {str(e)}"))

    if not frame_counts:
        raise ValueError("No valid videos found for frame count calculation")

    target_frames = int(np.mean(frame_counts))
    print(f"Target frames per video: {target_frames}")

    # Process each video
    print("\nProcessing videos...")
    for video_path in tqdm(video_files):
        try:
            # Create output path preserving folder structure
            relative_path = video_path.relative_to(input_folder)
            output_path = output_folder / relative_path
            output_path.parent.mkdir(parents=True, exist_ok=True)

            # Read video
            cap = cv2.VideoCapture(str(video_path))
            frames = []
            while cap.isOpened():
                ret, frame = cap.read()
                if not ret:
                    break
                frames.append(frame)
            cap.release()

            if not frames:
                failed_videos.append((video_path, "No frames could be read from video"))
                continue

            # Calculate motion scores to find start and end
            motion_scores = []
            for i in range(1, len(frames)):
                diff = cv2.absdiff(frames[i-1], frames[i])
                motion_scores.append(np.mean(diff))

            # Find meaningful motion
            threshold = np.mean(motion_scores) * 0.3
            start_idx = 0
            end_idx = len(frames) - 1

            # Find start of motion
            for i, score in enumerate(motion_scores):
                if score > threshold:
                    start_idx = max(0, i - 5)
                    break

            # Find end of motion
            for i in reversed(range(len(motion_scores))):
                if motion_scores[i] > threshold:
                    end_idx = min(len(frames), i + 5)
                    break

            # Trim frames
            frames = frames[start_idx:end_idx]

            # Standardize length
            if len(frames) > 0:
                indices = np.linspace(0, len(frames)-1, target_frames, dtype=int)
                frames = [frames[i] for i in indices]

                # Save processed video
                height, width = frames[0].shape[:2]
                out = cv2.VideoWriter(
                    str(output_path),
                    cv2.VideoWriter_fourcc(*'mp4v'),
                    30,
                    (width, height)
                )

                for frame in frames:
                    out.write(frame)
                out.release()

                successful_videos.append(video_path)
            else:
                failed_videos.append((video_path, "No frames remained after trimming"))

        except Exception as e:
            failed_videos.append((video_path, str(e)))

    # Print summary
    print("\nProcessing Summary:")
    print(f"Total videos: {len(video_files)}")
    print(f"Successfully processed: {len(successful_videos)}")
    print(f"Failed: {len(failed_videos)}")

    if failed_videos:
        print("\nFailed Videos:")
        for path, error in failed_videos:
            print(f"\nVideo: {path}")
            print(f"Error: {error}")

        # Save failed videos list to a file in the output folder
        log_path = output_folder / 'failed_videos_log.txt'
        with open(log_path, 'w') as f:
            f.write("Failed Videos Report\n")
            f.write("==================\n\n")
            for path, error in failed_videos:
                f.write(f"Video: {path}\n")
                f.write(f"Error: {error}\n\n")
        print(f"\nDetailed error log saved to: {log_path}")

    return output_folder, failed_videos, successful_videos

if __name__ == "__main__":
    # Input folder containing the videos
    PROJECT_FOLDER = '/content/drive/MyDrive/OMDENA/data_collected'
    # Base folder where the processed videos will be saved
    OUTPUT_BASE_FOLDER = '/content/drive/MyDrive/OMDENA'

    processed_folder, failed_videos, successful_videos = process_sign_language_videos(
        PROJECT_FOLDER,
        OUTPUT_BASE_FOLDER
    )

Found 267 videos
Calculating average video length...


100%|██████████| 267/267 [04:36<00:00,  1.04s/it]


Target frames per video: 113

Processing videos...


100%|██████████| 267/267 [13:03<00:00,  2.94s/it]


Processing Summary:
Total videos: 267
Successfully processed: 267
Failed: 0


# Enhancement:

key points for the video enhancement step:
Enhances processed videos by:


*   Improves image quality using CLAHE (Contrast Limited Adaptive Histogram Equalization) on LAB color space
*   Applies light sharpening with custom kernel matrix for detail enhancement
*   Optional denoising for noise reduction (disabled by default for speed)
*   Saves enhanced results in 'enhanced_videos_v2' using GPU acceleration and chunk processing


In [4]:
import cv2
import numpy as np
from pathlib import Path
from tqdm import tqdm
import torch
import gc
import os

def enhance_frame(frame, apply_denoise=False):
    """Helper function to enhance a single frame"""
    try:
        # Convert to LAB color space
        lab = cv2.cvtColor(frame, cv2.COLOR_BGR2LAB)
        l, a, b = cv2.split(lab)

        # Apply CLAHE to L channel
        clahe = cv2.createCLAHE(clipLimit=2.0, tileGridSize=(8,8))
        cl = clahe.apply(l)

        # Merge channels
        enhanced_lab = cv2.merge((cl,a,b))

        # Convert back to BGR
        enhanced_bgr = cv2.cvtColor(enhanced_lab, cv2.COLOR_LAB2BGR)

        # Optional denoising (disabled by default to improve speed)
        if apply_denoise:
            enhanced_bgr = cv2.fastNlMeansDenoisingColored(enhanced_bgr, None, 5, 5, 3, 11)

        # Light sharpening
        kernel = np.array([[-0.5,-0.5,-0.5],
                         [-0.5, 5,-0.5],
                         [-0.5,-0.5,-0.5]])
        sharpened = cv2.filter2D(enhanced_bgr, -1, kernel)

        return sharpened

    except Exception as e:
        print(f"Error enhancing frame: {e}")
        return frame

def process_video_chunk(args):
    """Process a chunk of videos to allow for better memory management"""
    video_paths, processed_folder, output_base_folder, start_idx = args
    failed_videos = []
    successful_videos = []

    enhanced_folder = Path(output_base_folder) / 'enhanced_videos_v2'

    for video_path in tqdm(video_paths, desc="Processing videos in chunk"):
        try:
            # Create output path preserving folder structure
            relative_path = video_path.relative_to(processed_folder)
            output_path = enhanced_folder / relative_path
            output_path.parent.mkdir(parents=True, exist_ok=True)

            # Read video
            cap = cv2.VideoCapture(str(video_path))

            if not cap.isOpened():
                raise Exception("Could not open video file")

            # Get video properties
            width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
            height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
            fps = int(cap.get(cv2.CAP_PROP_FPS))
            total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))

            # Create video writer
            fourcc = cv2.VideoWriter_fourcc(*'mp4v')
            out = cv2.VideoWriter(
                str(output_path),
                fourcc,
                fps,
                (width, height)
            )

            # Process each frame with progress bar
            for _ in range(total_frames):
                ret, frame = cap.read()
                if not ret:
                    break

                # Enhance frame (denoise disabled for speed)
                enhanced_frame = enhance_frame(frame, apply_denoise=False)

                # Write enhanced frame
                out.write(enhanced_frame)

            # Release resources
            cap.release()
            out.release()

            successful_videos.append(video_path)

        except Exception as e:
            failed_videos.append((video_path, str(e)))

        # Clear GPU memory after each video
        if torch.cuda.is_available():
            torch.cuda.empty_cache()
        gc.collect()

    return failed_videos, successful_videos

def enhance_videos_with_gpu(processed_folder, output_base_folder, chunk_size=5):
    """
    Enhanced version with GPU optimization and chunking
    """
    processed_folder = Path(processed_folder)
    output_base_folder = Path(output_base_folder)
    enhanced_folder = output_base_folder / 'enhanced_videos_v2'
    enhanced_folder.mkdir(exist_ok=True)

    # Get all video files
    video_files = list(processed_folder.rglob('*.mp4'))
    total_videos = len(video_files)
    print(f"Found {total_videos} videos to process")

    # Process in chunks to manage memory
    all_failed_videos = []
    all_successful_videos = []

    # Calculate number of chunks
    num_chunks = (total_videos + chunk_size - 1) // chunk_size

    # Process videos in chunks
    for chunk_idx in tqdm(range(num_chunks), desc="Processing chunks"):
        start_idx = chunk_idx * chunk_size
        end_idx = min(start_idx + chunk_size, total_videos)
        chunk_videos = video_files[start_idx:end_idx]

        print(f"\nProcessing chunk {chunk_idx + 1}/{num_chunks} ({len(chunk_videos)} videos)")

        # Process the chunk
        failed_chunk, successful_chunk = process_video_chunk((
            chunk_videos, processed_folder, output_base_folder, start_idx
        ))

        all_failed_videos.extend(failed_chunk)
        all_successful_videos.extend(successful_chunk)

        # Clear memory after each chunk
        if torch.cuda.is_available():
            torch.cuda.empty_cache()
        gc.collect()

        # Save progress after each chunk
        save_progress(enhanced_folder, all_failed_videos, all_successful_videos, chunk_idx)

    # Print final summary
    print("\nProcessing Complete!")
    print(f"Total videos processed: {total_videos}")
    print(f"Successfully processed: {len(all_successful_videos)}")
    print(f"Failed: {len(all_failed_videos)}")

    if all_failed_videos:
        save_error_log(enhanced_folder, all_failed_videos)

    return enhanced_folder, all_failed_videos, all_successful_videos

def save_progress(folder, failed, successful, chunk_idx):
    """Save progress after each chunk"""
    progress_file = folder / f'progress_chunk_{chunk_idx}.txt'
    with open(progress_file, 'w') as f:
        f.write(f"Processed through chunk {chunk_idx}\n")
        f.write(f"Successful so far: {len(successful)}\n")
        f.write(f"Failed so far: {len(failed)}\n")

def save_error_log(folder, failed_videos):
    """Save detailed error log"""
    log_path = folder / 'failed_enhancements_log.txt'
    with open(log_path, 'w') as f:
        f.write("Failed Enhancements Report\n")
        f.write("========================\n\n")
        for path, error in failed_videos:
            f.write(f"Video: {path}\n")
            f.write(f"Error: {error}\n\n")

if __name__ == "__main__":
    # Check for GPU
    if torch.cuda.is_available():
        print(f"Using GPU: {torch.cuda.get_device_name(0)}")
    else:
        print("No GPU available, using CPU")

    # Base paths - convert string paths to Path objects
    BASE_FOLDER = Path('/content/drive/MyDrive/OMDENA')
    PROCESSED_FOLDER = BASE_FOLDER / 'processed_videos_v2'

    # Reduced chunk size for better memory management
    CHUNK_SIZE = 5

    # Enhance the videos
    enhanced_folder, failed_videos, successful_videos = enhance_videos_with_gpu(
        PROCESSED_FOLDER,
        BASE_FOLDER,
        chunk_size=CHUNK_SIZE
    )

Using GPU: Tesla T4
Found 267 videos to process


Processing chunks:   0%|          | 0/54 [00:00<?, ?it/s]


Processing chunk 1/54 (5 videos)



Processing chunks:   2%|▏         | 1/54 [00:30<27:15, 30.85s/it]


Processing chunk 2/54 (5 videos)


...


Processing chunk 53/54 (5 videos)



Processing chunks:  98%|█████████▊| 53/54 [31:30<00:36, 36.48s/it]


Processing chunk 54/54 (2 videos)



Processing chunks: 100%|██████████| 54/54 [31:37<00:00, 35.13s/it]


Processing Complete!
Total videos processed: 267
Successfully processed: 267
Failed: 0


In [5]:
def cleanup_progress_files(enhanced_folder):
    """
    Clean up progress and log files after successful processing
    """
    enhanced_folder = Path(enhanced_folder)

    # Delete all progress chunk files
    progress_files = list(enhanced_folder.glob('progress_chunk_*.txt'))
    for file in progress_files:
        try:
            file.unlink()
            print(f"Deleted: {file}")
        except Exception as e:
            print(f"Could not delete {file}: {e}")

    # Delete the error log file if it exists
    error_log = enhanced_folder / 'failed_enhancements_log.txt'
    if error_log.exists():
        try:
            error_log.unlink()
            print(f"Deleted: {error_log}")
        except Exception as e:
            print(f"Could not delete {error_log}: {e}")

    print("Cleanup complete!")

# Use it like this:
if __name__ == "__main__":
    BASE_FOLDER = Path('/content/drive/MyDrive/OMDENA')
    enhanced_folder = BASE_FOLDER / 'enhanced_videos_v2'
    cleanup_progress_files(enhanced_folder)

Deleted: /content/drive/MyDrive/OMDENA/enhanced_videos_v2/progress_chunk_0.txt
Deleted: /content/drive/MyDrive/OMDENA/enhanced_videos_v2/progress_chunk_1.txt
Deleted: /content/drive/MyDrive/OMDENA/enhanced_videos_v2/progress_chunk_2.txt
Deleted: /content/drive/MyDrive/OMDENA/enhanced_videos_v2/progress_chunk_3.txt
Deleted: /content/drive/MyDrive/OMDENA/enhanced_videos_v2/progress_chunk_4.txt
Deleted: /content/drive/MyDrive/OMDENA/enhanced_videos_v2/progress_chunk_5.txt
Deleted: /content/drive/MyDrive/OMDENA/enhanced_videos_v2/progress_chunk_6.txt
Deleted: /content/drive/MyDrive/OMDENA/enhanced_videos_v2/progress_chunk_7.txt
Deleted: /content/drive/MyDrive/OMDENA/enhanced_videos_v2/progress_chunk_8.txt
Deleted: /content/drive/MyDrive/OMDENA/enhanced_videos_v2/progress_chunk_9.txt
Deleted: /content/drive/MyDrive/OMDENA/enhanced_videos_v2/progress_chunk_10.txt
Deleted: /content/drive/MyDrive/OMDENA/enhanced_videos_v2/progress_chunk_11.txt
Deleted: /content/drive/MyDrive/OMDENA/enhanced_vi